# GRU LSTM
Attempts to create a GRU LSTM model to use for Yelp Dataset Challenge

In [ ]:
%matplotlib inline

import pandas as pd
import sklearn as sk
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import itertools

import utils

import sys
import argparse
import random

## gru_lstm class
...

In [ ]:
# Uses tf.contrib to generate GRU cell
# Saves implementing own GRUCell from scratch
# Based off of PTB example: # https://github.com/tensorflow/models/blob/master/tutorials/rnn/ptb/ptb_word_lm.py

def data_type():
    """
    Good for modying data type needed instead of having to change every single one
    """
    return tf.float32

class gru_lstm(object):
    def __init__(self, X_inputs, y_labels, num_steps, batch_size, is_training, 
                 vocab_size, embedding_size, state_size,, num_layers):
        
        self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.state_size = hidden_units
        self.batch_size = batch_size
        self.num_layers = num_layers
        self._is_training = is_training

        self._cell = None
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        embedding = tf.get_variable("embedding", [vocab_size, size], dtype=data_type())
        
        inputs = tf.nn.embedding_lookup(embedding)
        if is_training and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)
        
        # Output is Neural Network
        output, state = self.stacked_gru_graph(X_inputs, state_size, keep_prob, num_layers, is_training)
        softmax_w = tf.get_variable("softmax_w", [size, vocab_size], dtype=data_type())
        softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=data_type())
        
        
        logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
        
        ###
        # seq2seq.sequence_loss is weighted cross entropy over sequence of logits
        ###
        loss = tf.contrib.seq2seq.sequence_loss(logits,
                                                y_labels,
                                                tf.ones([self.batch_size, self.num_steps], dtype=data_type()),
                                                average_across_timesteps=False,
                                                average_across_batch=True)
                                               
        
        self._cost = tf.reduce_sum(loss)
        self._final_state = state
        
        self._learn_rate = tf.Variable(0.0, trainable=False) # Learning Rate
        gradients, _ = tf.clip_by_global_norm(tf.gradients(self._cost, tvars),
                                          config.max_grad_norm)
        train_vars = tf.trainable_variables() # Returns all variables specified trainable=True
        
        optimizer = tf.train.RMSPropOptimizer(self._learn_rate) # Using RMSProp for RNN's rather than SGD as usual
        self._train_op = optimizer.apply_gradients(zip(gradients, train_vars),
                                                   global_step=tf.train.get_or_create_global_step())
        
        self._new_learn_rate = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
        self._learn_update = tf.assign(self._learn_rate, self._new_learn_rate)
    
    def gru_cell(self, hidden_state_size, keep_prob, is_training, layer, num_layers):
    
        assert isinstance(keep_prob, float)
        assert isinstance(hidden_state_size, int)
        assert isinstance(is_training, bool)
        assert isinstance(layer, int)
        assert isinstance(num_layers, int)
        
        if num_layers <= 0:
            raise AssertionError("Need positive number of layers")
    
        cell = tf.contrib.rnn.GRUCell(hidden_state_size)
    
        if layer == num_layers - 1:
            return tf.contrib.rnn.OutputProjectionWrapper(cell,
                                                          output_size=6)
        elif is_training and keep_prob < 1:
            return tf.contrib.rnn.DropoutWrapper(cell,
                                                 output_keep_prob=keep_prob)
        else:
            return cell
        
    def stacked_gru_graph(self, X_inputs, state_size, keep_prob, num_layers, is_training):
        
        assert isinstance(state_size, int)
        assert isinstance(keep_prob, float)
        if keep_prob < 0 or keep_prob > 1:
            raise AssertionError("needs to be a value between 0 and 1")
        assert isinstance(num_layers, int)
        assert isinstance(is_training, bool)
        
        stacked_gru = tf.contrib.rnn.MultiRNNCell(
            cells=[gru_cell(state_size, keep_prob, _, num_layers, is_training) for _ in range(num_layers)], 
            state_is_tupl=True)
        
        inputs = tf.unstack(X_inputs, num=num_steps, axis=1)
        
        self._initial_state = cell.zero_state(config.batch_size, data_type())
        
        outputs, state = tf.contrib.rnn.static_rnn(cell, inputs,
                                                   initial_state=self._initial_state)
        
        return output, state
    
    def assign_lr(self, session, lr_value):
        """
        :param: lr_value- new learning rate value multiplied by decay rate
        """
        assert isinstance(lr_value, float)
        session.run(self._learn_update, feed_dict={self._new_learn_rate: lr_value})

In [ ]:
num_layers = 2
state_size = 1024
batch_size = 128
time_steps = 2
num_features = 100
keep_prob = 0.8 # Keep 80% if outputs after when implementing dropout

# Based off of TensorFlow PTB example found at 
# https://github.com/tensorflow/models/blob/master/tutorials/rnn/ptb/ptb_word_lm.py

words_in_dataset = tf.placeholder(tf.float32, [time_steps, batch_size, num_features])




# Initial state of LSTM
# initial_state = state = stacked_gru.zero_state(batch_size, tf.float32)
